In [17]:
from utils.data_preprocess import load_data, split_xy_save, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
from utils.model_evaluation import plot_test_pred
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [18]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

single_leakage, _ = load_data(total_samples = cfg['experiment']['total_samples'])
data = single_leakage.drop(columns=["mfc10_residual"], inplace=False)


In [19]:
y = data[['x1', 'y1']]
x = data.drop(['x1', 'y1'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 

scaler_flows = StandardScaler()
X_train_sc = scaler_flows.fit_transform(X_train)
X_test_sc = scaler_flows.transform(X_test)
X_val_sc = scaler_flows.transform(X_val)

scaler_coords = StandardScaler()
y_train_sc = scaler_coords.fit_transform(y_train)
y_test_sc = scaler_coords.transform(y_test)
y_val_sc = scaler_coords.transform(y_val)

In [20]:
def my_loss_fn(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError(y_true[:,0:1], y_pred[:,0:1])
    mae = tf.keras.losses.MeanAbsoluteError(y_true[:,-1], y_pred[:,-1])
    loss = mse + mae
    return loss

In [21]:
linear_model = keras.Sequential([layers.Dense(256), layers.Dense(2)])
# linear_model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.01), metrics='mae')
linear_model.compile(
#     loss='mean_squared_error',
    loss = my_loss_fn,
    optimizer=tf.keras.optimizers.Nadam(0.01), metrics='mae')

history = linear_model.fit(
        X_train,
        y_train,
        epochs=100,
        # Suppress logging.
        verbose=1,
        validation_data = (X_val, y_val))

model_evaluate, y_pred = model_eval(linear_model, X_test, y_test, X_train, y_train, X_val, y_val)

Epoch 1/100


TypeError: in user code:

    File "/home/vn/miniconda3/envs/studiarbeit/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_90446/929786181.py", line 2, in my_loss_fn  *
        mse = tf.keras.losses.MeanSquaredError(y_true[:,0:1], y_pred[:,0:1])
    File "/home/vn/miniconda3/envs/studiarbeit/lib/python3.9/site-packages/keras/losses.py", line 358, in __init__  **
        super().__init__(mean_squared_error, name=name, reduction=reduction)
    File "/home/vn/miniconda3/envs/studiarbeit/lib/python3.9/site-packages/keras/losses.py", line 246, in __init__
        super().__init__(reduction=reduction, name=name)
    File "/home/vn/miniconda3/envs/studiarbeit/lib/python3.9/site-packages/keras/losses.py", line 83, in __init__
        losses_utils.ReductionV2.validate(reduction)
    File "/home/vn/miniconda3/envs/studiarbeit/lib/python3.9/site-packages/keras/utils/losses_utils.py", line 87

In [ ]:
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
sns.set_context('notebook')
history_frame= pd.DataFrame(data= history.history)
history_frame.plot(figsize=(8,5))
plt.show

In [ ]:
plot_test_pred(y_test, y_pred, scaler_coords)

In [ ]:
t1 = tf.constant([[0, 1], [2, 3], [4, 5], [6, 7]])
print(t1)

print(t1[:,-1])

tf.Tensor(
[[0 1]
 [2 3]
 [4 5]
 [6 7]], shape=(4, 2), dtype=int32)
tf.Tensor([1 3 5 7], shape=(4,), dtype=int32)
